In [1]:
from skimage.metrics import structural_similarity as ssim
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from shutil import copyfile
from tensorflow.keras.callbacks import Callback
from ipywidgets import interact, widgets
import time
from tqdm import tqdm

In [2]:
def fit_rotated_ellipse_ransac(data,iter=50,sample_num=10,offset=80.0):

    count_max = 0
    effective_sample = None

    for i in range(iter):
        sample = np.random.choice(len(data), sample_num, replace=False)

        xs = data[sample][:,0].reshape(-1,1)
        ys = data[sample][:,1].reshape(-1,1)

        J = np.mat( np.hstack((xs*ys,ys**2,xs, ys, np.ones_like(xs,dtype=np.float64))) )
        Y = np.mat(-1*xs**2)
        P= (J.T * J).I * J.T * Y

        # fitter a*x**2 + b*x*y + c*y**2 + d*x + e*y + f = 0
        a = 1.0; b= P[0,0]; c= P[1,0]; d = P[2,0]; e= P[3,0]; f=P[4,0];
        ellipse_model = lambda x,y : a*x**2 + b*x*y + c*y**2 + d*x + e*y + f

        # threshold 
        ran_sample = np.array([[x,y] for (x,y) in data if np.abs(ellipse_model(x,y)) < offset ])

        if(len(ran_sample) > count_max):
            count_max = len(ran_sample) 
            effective_sample = ran_sample

    return fit_rotated_ellipse(effective_sample)


def fit_rotated_ellipse(data):

    xs = data[:,0].reshape(-1,1) 
    ys = data[:,1].reshape(-1,1)

    J = np.mat( np.hstack((xs*ys,ys**2,xs, ys, np.ones_like(xs,dtype=np.float64))) )
    Y = np.mat(-1*xs**2)
    P= (J.T * J).I * J.T * Y

    a = 1.0; b= P[0,0]; c= P[1,0]; d = P[2,0]; e= P[3,0]; f=P[4,0];
    theta = 0.5* np.arctan(b/(a-c))  
    
    cx = (2*c*d - b*e)/(b**2-4*a*c)
    cy = (2*a*e - b*d)/(b**2-4*a*c)

    cu = a*cx**2 + b*cx*cy + c*cy**2 -f
    w= np.sqrt(cu/(a*np.cos(theta)**2 + b* np.cos(theta)*np.sin(theta) + c*np.sin(theta)**2))
    h= np.sqrt(cu/(a*np.sin(theta)**2 - b* np.cos(theta)*np.sin(theta) + c*np.cos(theta)**2))

    ellipse_model = lambda x,y : a*x**2 + b*x*y + c*y**2 + d*x + e*y + f

    error_sum = np.sum([ellipse_model(x,y) for x,y in data])
    print('fitting error = %.3f' % (error_sum))

    return (cx,cy,w,h,theta)

def apply_blur(image, blur_amount=5):
    return cv2.GaussianBlur(image, (blur_amount, blur_amount), 0)

In [3]:
def fitPupil(image,circ_thresh=0.5,thresh_val=60,kernel=cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3)),min_thresh=170,max_thresh=280):
        #print(image)
        
        temp_image = image.copy()
       
        inf_img = temp_image.copy()
        
        image_gray = cv2.cvtColor(temp_image , cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(image_gray,(3,3),0)
        
        ret,thresh1 = cv2.threshold(blur,thresh_val,255,cv2.THRESH_BINARY)
        
        opening = cv2.morphologyEx(thresh1, cv2.MORPH_OPEN, kernel)
        
        closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
        
        
        #size_el = 0
        temp_image  = 255 - closing
        
        
        contours, hierarchy = cv2.findContours(temp_image , cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        hull = []
        for i in range(len(contours)):
          hull.append(cv2.convexHull(contours[i], False)) 
        
        cx,cy,w,h,theta = 0.0,0.0,0.0,0.0,0.0
        for con in hull:
            
            approx = cv2.approxPolyDP(con, 0.01 * cv2.arcLength(con,True),True)
            area = cv2.contourArea(con)
            perimeter = cv2.arcLength(con, True)
            circularity = 4 * np.pi * (area / (perimeter * perimeter))


            if circularity > circ_thresh:
                
            
        
        
                if(len(approx) > 10 and area > 70):
                        try:
                        
                            cx,cy,w,h,theta = fit_rotated_ellipse_ransac(con.reshape(-1,2))
                            #size_el = ellipse_circumference(w,h)

                            # if size_el > min_thresh and size_el < max_thresh:
                            
                            #xcoordinates.append(cx)
                            #ycoordinates.append(cy)
                            cv2.ellipse(inf_img,(int(cx),int(cy)),(int(w),int(h)),theta*180.0/np.pi,0.0,360.0,(0,255,255),1)
                            inf_img = cv2.drawMarker(inf_img, (int(cx),int(cy)),(0, 255, 255),cv2.MARKER_CROSS,2,1)
                            #else: cx,cy,w,h,theta = 0.0,0.0,0.0,0.0,0.0
                            
                        except Exception as e: pass
            
        return inf_img,[cx,cy,w,h,theta]
    

In [45]:
def draw_text(frame, text_lines, position):
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.5
    font_thickness = 1
    color = (255, 255, 255)  # White color in BGR
    
    for i, line in enumerate(text_lines):
        y_offset = i * 20  # Adjust vertical offset
        cv2.putText(frame, line, (position[0], position[1] + y_offset), font, font_scale, color, font_thickness, cv2.LINE_AA)



In [49]:
# Function to calculate distance between two points
def distance(point1, point2):
    return np.sqrt((point2[0] - point1[0])**2 + (point2[1] - point1[1])**2)


In [121]:

def calculate_velocity(object_data, prev_object_data, frame_rate):
  """
  This function calculates the velocity of an object in pixels per frame.

  Args:
      object_data: A dictionary containing the current object data (x, y, width, height, angle).
      prev_object_data: A dictionary containing the previous object data (from the previous frame).
      frame_rate: The frame rate of the video.

  Returns:
      A dictionary containing the velocity in x and y directions (vx, vy) in pixels per frame.
  """
  # Check if previous data is available
  if not prev_object_data:
    return {"vx": 0, "vy": 0}

  center_x = int(object_data[0] + object_data[2] / 2)
  center_y = int(object_data[1] + object_data[3] / 2)
  prev_center_x = int(prev_object_data[0] + prev_object_data[2] / 2)
  prev_center_y = int(prev_object_data[1] + prev_object_data[3] / 2)

  # Calculate velocity in x and y directions (pixels per frame)
  vx = (center_x - prev_center_x) / frame_rate
  vy = (center_y - prev_center_y) / frame_rate

  return {"vx": vx, "vy": vy}

In [115]:
cap.release()

In [106]:
prev_object_data = None

In [122]:
def draw_velocity_arrow(frame, object_data, velocity, scale=10):
  """
  This function draws an arrow on a frame to represent the object's velocity.

  Args:
      frame: The frame image from the video.
      object_data: A dictionary containing object data (x, y, width, height, angle).
      velocity: A dictionary containing velocity data (vx, vy) in pixels per frame.
      scale: A factor to scale the arrow length for better visualization (optional).
  """
  if object_data and velocity:
    # Calculate arrow end point based on object center and scaled velocity
    arrow_x = int(object_data[0] + velocity["vx"] * scale)
    arrow_y = int(object_data[1] + velocity["vy"] * scale)
    end_point = (arrow_x, arrow_y)

    print(velocity)

    print(object_data[0], object_data[1])

    print(end_point)

    # Draw arrow line using cv2.arrowedLine
    cv2.arrowedLine(frame, (int(object_data[0]), int(object_data[1])), end_point,
                    (0, 0, 255), 2)  # Color: Blue

In [134]:
cap = cv2.VideoCapture('./11.avi')

frame_rate = cap.get(cv2.CAP_PROP_FPS)
while(cap.isOpened()):
  # Capture frame-by-frame
    ret, frame = cap.read()
    
    
    if ret == True:
        frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)
        
        

        out,cnt = fitPupil(frame,circ_thresh=.5,thresh_val=45)

        if all(element != 0 for element in cnt):
            x,y,w,h,angle = cnt
            
            
            # You can adjust the position as needed
            object_data = cnt

            velocity = calculate_velocity(object_data, prev_object_data, frame_rate)
            
            

           
            text_lines = [f'x: {cnt[0]:.2f}', f'y: {cnt[1]:.2f}', f'w: {cnt[2]:.2f}', f'h: {cnt[3]:.2f}',
                           f'angle: {cnt[4]:.2f}',f'vx: {velocity["vx"]:.2f}',
                           f'vy: {velocity["vy"]:.2f}'
                           ]
            draw_text(out, text_lines, (10, 30))
            if velocity:
              #pass
              draw_velocity_arrow(out,object_data,velocity)
            cv2.imshow("Frame",out)    
            
            prev_object_data = object_data
            if cv2.waitKey(100) & 0xFF == ord('q'):
                cap.release()
                cv2.destroyAllWindows()
                break   
    # Break the loop
    else: break
cap.release()
cv2.destroyAllWindows()

fitting error = 0.000
{'vx': -0.94, 'vy': -0.96}
66.97404877437832 116.99331320564012
(57, 107)
fitting error = -0.000
{'vx': 0.7, 'vy': 0.24}
98.12479090510932 126.92954054648139
(105, 129)
fitting error = 0.000
{'vx': 0.0, 'vy': -0.08}
97.21035393145272 124.31881351901389
(97, 123)
fitting error = 0.000
{'vx': 0.0, 'vy': 0.06}
97.7052501020399 126.14576108557084
(97, 126)
fitting error = 0.000
{'vx': 0.0, 'vy': 0.0}
97.40785693940306 125.65936730712517
(97, 125)
fitting error = 0.000
{'vx': 0.02, 'vy': -0.02}
97.98349419210331 125.20324075604407
(98, 125)
fitting error = 0.000
{'vx': -0.62, 'vy': 0.0}
70.88115508191662 127.21292271607192
(64, 127)
fitting error = 0.000
{'vx': 0.06, 'vy': 0.02}
71.41265123159204 127.40114202375857
(72, 127)
fitting error = 0.000
{'vx': -0.08, 'vy': -0.32}
68.01829217547964 110.33398143552138
(67, 107)
fitting error = 0.000
{'vx': 0.0, 'vy': 0.02}
67.56892329234346 110.97107742259783
(67, 111)
fitting error = 0.000
{'vx': 0.0, 'vy': 0.02}
67.1682719601

In [65]:
# Example usage
object_data = [100,50,40,30,30]
prev_object_data = [90,40,40,30,30]  # No data from previous frame initially
frame_rate = 30  # Assuming video frame rate is 30 fps

velocity = calculate_velocity(object_data, prev_object_data, frame_rate)

if velocity:
  print("Velocity:", velocity)  # Output: {"vx": 5.0, "vy": 2.0} (assuming movement)
else:
  print("Not enough data to calculate velocity.")

# Update previous object data for future calculations
prev_object_data = object_data

Velocity: {'vx': 0.3333333333333333, 'vy': 0.3333333333333333}
